In [ ]:

# # Analisi reti sociali - Matrici di adiacenza
# Questo notebook crea matrici di adiacenza pesate e binarie a partire da un file Excel contenente risposte a un questionario SNA.

import pandas as pd

# 📥 1. Carica il file Excel
from google.colab import files
uploaded = files.upload()

# ⬇️ 2. Leggi il file Excel
file_name = list(uploaded.keys())[0]
df = pd.read_excel(file_name)

# 🧹 3. Pulizia colonne
df = df.rename(columns={
    "Cod profilo rispondente": "from",
    "Codice domanda": "question",
    "Cod profilo valutato": "to",
    "Risposta numerica": "score"
})
df["from"] = df["from"].str.lower()
df["to"] = df["to"].str.lower()

# 🧠 4. Definizione domande per rete
domande_reti = {
    "CC": [f"CC_0{i}" for i in range(1, 5)],
    "EC": [f"EC_0{i}" for i in range(1, 6)],
    "CM": [f"CM_0{i}" for i in range(1, 3)],
    "FR": ["FR_01"]
}
nodes = sorted(df["from"].unique())

# ⚙️ 5. Funzione per creare matrici
def crea_matrici(df, domande, is_binaria=False, soglia=2.5):
    df_rete = df[df["question"].isin(domande)]
    media_pesi = df_rete.groupby(["from", "to"])["score"].mean().reset_index()

    matrice_pesata = pd.DataFrame(0.0, index=nodes, columns=nodes)
    for _, row in media_pesi.iterrows():
        matrice_pesata.loc[row["from"], row["to"]] = row["score"]

    if is_binaria:
        matrice_binaria = matrice_pesata.copy()
    else:
        matrice_binaria = (matrice_pesata >= soglia).astype(int)

    return matrice_pesata, matrice_binaria

# 🧾 6. Costruzione e salvataggio delle matrici
matrici = {}
for rete, domande in domande_reti.items():
    is_binaria = (rete == "FR")
    pesata, binaria = crea_matrici(df, domande, is_binaria)
    matrici[rete] = {
        "pesata": pesata,
        "binaria": binaria
    }

# 💾 7. Esporta tutte le matrici in un file Excel
output_file = "matrici_adiacenza_reti.xlsx"
with pd.ExcelWriter(output_file) as writer:
    for rete, mat in matrici.items():
        mat["pesata"].to_excel(writer, sheet_name=f"{rete}_pesata")
        mat["binaria"].to_excel(writer, sheet_name=f"{rete}_binaria")

# 📤 8. Download del file finale
files.download(output_file)
